In [2]:
def createSeeds(seed, n):
    seeds = []
    for i in range(n):
        seeds.append(seed)
        seed = (seed * 5) % 1
    return seeds

In [46]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)
def calculaResultados(lista, capacidade):
    data_array = np.array(lista)

    # Calculate the mean of each column
    column_means = np.mean(data_array, axis=0)

    resultados = []
    for i in range(capacidade + 1):
        resultado = [i, column_means[i + 1], (column_means[i + 1] / column_means[0]) * 100]
        resultados.append(resultado)
    resultados.append(['TOTAL', column_means[0], 100])
    return resultados

In [47]:
class FilaTandem:

    def __init__(self, config) -> None:
        self.FILA1 = 0
        self.FILA2 = 0
        self.tempo = 0.0
        self.fila_1_arrival_limits = config['fila_1_arrival_limits']
        self.fila_1_service_limits = config['fila_1_service_limits']
        self.fila_2_service_limits = config['fila_2_service_limits']
        self.seeds = config['seeds']
        self.seed = 0
        self.fila_1_capacidade = config['fila_1_capacidade']
        self.fila_2_capacidade = config['fila_2_capacidade']
        self.fila_1_servidores = config['fila_1_servidores']
        self.fila_2_servidores = config['fila_2_servidores']

    def escalonador(self, tempo_inicial):
        self.escalonador_eventos = []
        self.tabela_estados_fila1 = []
        self.tabela_estados_fila2 = []
        # cria um vetor estado inicial com tamanho 3 + (capacidade + 1)
        fila_1_estado = [0.0] * (4 + (self.fila_1_capacidade + 1))
        fila_1_estado[0] = None
        fila_1_estado[1] = self.FILA1
        fila_1_estado[2] = self.FILA2
        fila_1_estado[3] = self.tempo
        self.tabela_estados_fila1.append(fila_1_estado)
        
        fila_2_estado = [0.0] * (4 + (self.fila_2_capacidade + 1))
        fila_2_estado[0] = None
        fila_2_estado[1] = self.FILA1
        fila_2_estado[2] = self.FILA2
        fila_2_estado[3] = self.tempo
        self.tabela_estados_fila2.append(fila_2_estado)
        
        # novo evento sem sorteio
        evento = ['CH1', tempo_inicial, 0.0]
        self.escalonador_eventos.append(evento)
        while self.seed < len(self.seeds):
            self.escalonador_eventos.sort( key=lambda x: x[1] )
            evento = self.escalonador_eventos.pop(0) 
            tempo_anterior = self.tempo
            self.tempo = evento[1]
            if evento[0] == 'CH1':
                if self.FILA1 < self.fila_1_capacidade:
                    self.FILA1 += 1
                    if self.FILA1 <= self.fila_1_servidores:
                        self.agendaP12(self.tempo)
                self.agendaCH1(self.tempo)
            elif evento[0] == 'SA2':
                self.FILA2 -= 1
                if self.FILA2 > 0:
                    self.agendaSA2(self.tempo)
            elif evento[0] == 'P12':
                self.FILA1 -= 1
                if self.FILA1 >= self.fila_1_servidores:
                    self.agendaP12(self.tempo)
                if self.FILA2 < self.fila_2_capacidade:
                    self.FILA2 += 1
                    if self.FILA2 <= self.fila_2_servidores:
                        self.agendaSA2(self.tempo)       
            
            self.calculaEstado(evento, tempo_anterior)
        return self.tabela_estados_fila1[-1], self.tabela_estados_fila2[-1]

    def calculaEstado(self, evento, tempo_anterior):
        fila_1_ultimo_estado = self.tabela_estados_fila1[-1]
        fila_2_ultimo_estado = self.tabela_estados_fila2[-1]
        fila_1_ultima_fila = fila_1_ultimo_estado[1]
        fila_2_ultima_fila = fila_2_ultimo_estado[2]
    
        fila_1_novo_estado = fila_1_ultimo_estado.copy()
        fila_2_novo_estado = fila_2_ultimo_estado.copy()
        
        fila_1_novo_estado[0] = evento[0]
        fila_1_novo_estado[1] = self.FILA1
        fila_1_novo_estado[2] = self.FILA2
        fila_1_novo_estado[3] = self.tempo
        fila_2_novo_estado[0] = evento[0]
        fila_2_novo_estado[1] = self.FILA1
        fila_2_novo_estado[2] = self.FILA2
        fila_2_novo_estado[3] = self.tempo        
        # diferenca entre tempo atual e o anterior
        diferenca = self.tempo - tempo_anterior
        
        # atualiza quanto tempo a fila ficou com a quantidade de pessoas anterior
        # diferenca + tempo anterior da fila com a quantidade de pessoas anterior
        fila_1_novo_estado[4 + fila_1_ultima_fila] = diferenca + fila_1_ultimo_estado[4 + fila_1_ultima_fila]
        fila_2_novo_estado[4 + fila_2_ultima_fila] = diferenca + fila_2_ultimo_estado[4 + fila_2_ultima_fila]
        
        self.tabela_estados_fila1.append(fila_1_novo_estado)
        self.tabela_estados_fila2.append(fila_2_novo_estado)

    def _sorteio(self, min_max):
        if self.seed == len(self.seeds):
            return 0
        self.seed += 1
        return (min_max[1] - min_max[0]) * self.seeds[self.seed - 1] + min_max[0]
    
    def agendaCH1(self, tempo):
        sorteio = self._sorteio(self.fila_1_arrival_limits)
        evento = ['CH1', tempo + sorteio, sorteio]
        self.escalonador_eventos.append(evento)
        
    def agendaSA2(self, tempo):
        sorteio = self._sorteio(self.fila_2_service_limits)
        evento = ['SA2', tempo + sorteio, sorteio]
        self.escalonador_eventos.append(evento)
        
    def agendaP12(self, tempo):
        sorteio = self._sorteio(self.fila_1_service_limits)
        evento = ['P12', tempo + sorteio, sorteio]
        self.escalonador_eventos.append(evento)

In [48]:
config = {
        'fila_1_arrival_limits':[1,3],
        'fila_1_service_limits':[2,4],
        'fila_2_service_limits':[1,3],
        'seeds':[0.9921, 0.0004, 0.5534, 0.2761, 0.3398, 0.8963, 0.9023, 0.0132, 0.4569, 0.5121, 0.9208, 0.0171, 0.2299, 0.8545, 0.6001, 0.2921],
        'fila_1_servidores':2,
        'fila_1_capacidade':3,
        'fila_2_servidores':1,
        'fila_2_capacidade':5
}

In [49]:
from tabulate import tabulate
import numpy as np

seeds = [0.9921, 0.0004, 0.5534, 0.2761, 0.3398]

fila_1_simulados = []
fila_2_simulados = []
for seed in seeds:
    seeds = createSeeds(seed, 100000)
    config['seeds'] = seeds
    filaTandem = FilaTandem(config)
    estados1, estados2 = filaTandem.escalonador(2.5000)
    fila_1_simulados.append(estados1[3:])
    fila_2_simulados.append(estados2[3:])

print('Fila 1')
print(tabulate(fila_1_simulados))

print('Fila 2')
print(tabulate(fila_2_simulados))

print('Fila 1 STATISTICS')
print(tabulate(calculaResultados(fila_1_simulados, config['fila_1_capacidade']), headers=['FILA 1', 'MEDIA', 'PERCENTUAL']))

print('Fila 2 STATISTICS')
print(tabulate(calculaResultados(fila_2_simulados, config['fila_2_capacidade']), headers=['FILA 2', 'MEDIA', 'PERCENTUAL']))


Fila 1
-------  -------  -------  -------  -------
67147.5  721.631  32044.1  31431.7  2950.06
50001.1    2.5    16666.7  33331.9     0
66905.3  692.632  31525.7  31662.7  3024.2
67026.9  690.818  31743.1  31658.4  2934.56
67129.5  700.358  32172.6  31301.3  2955.27
-------  -------  -------  -------  -------
Fila 2
-------  ---------  --------  -------  -------  -------  -------
67147.5  1199.12     9813.72  17119.4  18321    15922.4  4771.83
50001.1     4.5008  36770.8   13225.8      0        0       0
66905.3  1178.97    10032.2   17668.6  18062.3  15207.7  4755.5
67026.9  1223.41     9787.19  17630.3  18113.8  15452.5  4819.73
67129.5  1350.1     10660.4   18126.1  17868.8  14736.5  4387.5
-------  ---------  --------  -------  -------  -------  -------
Fila 1 STATISTICS
FILA 1        MEDIA    PERCENTUAL
--------  ---------  ------------
0           561.588      0.882416
1         28830.4       45.3009
2         31877.2       50.0883
3          2372.82       3.72838
TOTAL     63642

In [45]:
print('Fila 2')
print(tabulate(estados2, headers=['Evento', 'Fila 1', 'Fila 2', 'Tempo', '0', '1', '2', '3'], tablefmt='orgtbl'))

estados1_aux = []
for estado in estados1:
    estados1_aux.append(estado[3:])
    
estados2_aux = []
for estado in estados2:
    estados2_aux.append(estado[3:])

print('Fila 1 STATISTICS')
print(tabulate(calculaResultadosAux(estados1_aux, config['fila_1_capacidade'])))


Fila 2
| Evento   |   Fila 1 |   Fila 2 |   Tempo |      0 |      1 |      2 |      3 |
|----------+----------+----------+---------+--------+--------+--------+--------|
|          |        0 |        0 |  0      | 0      | 0      | 0      | 0      |
| CH1      |        1 |        0 |  2.5    | 2.5    | 0      | 0      | 0      |
| CH1      |        2 |        0 |  4.5004 | 4.5004 | 0      | 0      | 0      |
| CH1      |        3 |        0 |  6.7765 | 6.7765 | 0      | 0      | 0      |
| P12      |        2 |        1 |  7.4763 | 7.4763 | 0      | 0      | 0      |
| P12      |        1 |        2 |  8.1606 | 7.4763 | 0.6843 | 0      | 0      |
| CH1      |        2 |        2 |  9.1163 | 7.4763 | 0.6843 | 0.9557 | 0      |
| P12      |        1 |        3 | 11.1559 | 7.4763 | 0.6843 | 2.9953 | 0      |
| CH1      |        2 |        3 | 11.5732 | 7.4763 | 0.6843 | 2.9953 | 0.4173 |
| P12      |        1 |        3 | 12.1652 | 7.4763 | 0.6843 | 2.9953 | 1.0093 |
| SA2      |        1

In [44]:
print('Fila 2 STATISTICS')
print(tabulate(calculaResultadosAux(estados2_aux, config['fila_2_capacidade'])))

Fila 2 STATISTICS
-----  -------  ---------
0       7.4763   43.0948
1       0.6843    3.94443
2       7.6925   44.341
3       1.4954    8.61977
TOTAL  17.3485  100
-----  -------  ---------


In [43]:
def calculaResultadosAux(lista, capacidade):
    resultados = []
    for i in range(capacidade + 1):
        resultado = [i, lista[-1][i + 1], (lista[-1][i + 1] / lista[-1][0]) * 100]
        resultados.append(resultado)
    resultados.append(['TOTAL', lista[-1][0], 100])
    return resultados